In [14]:
from langchain_community.vectorstores import SKLearnVectorStore
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
import re

## Maestría en Inteligencia Artificial Aplicada (MNA)
### Proyecto Integrador
### Dra. Grettel Barceló Alonso / Dr. Carlos Alberto Villaseñor Padilla
### Avance 5. Implementación de la Base de Vectores Final

### Integrantes
- A01794457 - Iossif Moises Palli Laura
- A01793984 - Brenda Zurazy Rodríguez Pérez
- A01794630 - Jesús Ramseths Echeverría Rivera

### 1. Carga y división de documentos

In [15]:
name_files = ['Ley de Fondos de Inversión', 'Ley para Regular las Instituciones de Tecnología Financiera',
              'Ley para Regular las Agrupaciones Financieras'] # Nombre de los archivos

In [16]:
# Aplicación de Separador delimitado a 500 fragmentos
docs = []
for file in name_files:
    # Cargar un documento PDF y extraer sus páginas
    loader = PyPDFLoader(f'../base/{file}.pdf', extract_images=False)
    pages = loader.load()
    # Dividir el texto en fragmentos más pequeños
    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)
    tmp_docs = text_splitter.split_documents(pages)
    docs.extend(tmp_docs)

In [17]:
print(f'Cantidad de Documentos: {len(docs)}')

Cantidad de Documentos: 311


### 2. Preprocesamiento

In [18]:
# Limpiar el texto de los documentos
for doc in docs:
    # Se elimina espacios y saltos de línea al inicio y al final
    text = doc.page_content.strip()
    text = text.replace('\n', ' ')

    # Se wliminan espacios duplicados
    text = re.sub(r'[ \t]+', ' ', text)

    # Actualizar
    doc.page_content = text

# Filtrar documentos vacíos
docs = [doc for doc in docs if doc.page_content.strip() != '']

### 3. Creación de Base Vectorial

In [19]:
# Se usa este modelo por ser ligero en cuestión de recursos computacionales
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/multi-qa-MiniLM-L6-cos-v1")

In [20]:
# Definir la ruta donde se guardarán los embeddings
persist_path = "../database/embeddings_db"
# Especificación de la VectorDB
vector_store = SKLearnVectorStore.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_path=persist_path,
    serializer="parquet"
)
# Se crea la base
vector_store.persist()

### 4. Prueba de la Base Vectorial

In [21]:
# Se manda un prompt y se hace la búsqueda por similitud del coseno
prompt = "¿En cuál fecha fue publicada la Ley de Fondos de Inversión?"
documents = vector_store.similarity_search(prompt)

In [23]:
print('Resultado: \n')
print(documents[0].page_content)

Resultado: 

Para uso interno Para uso interno (3) LEY DE FONDOS DE INVERSIÓN Publicada en el Diario Oficial de la Federación el 4 de junio de 2001. Actualizada con las modificaciones publicadas en el propio Diario el 28 de junio de 2007, 10 d e enero y 13 de junio de 2014 , 20 de mayo de 2021 , 28 de diciembre de 2023 y 24 de enero de 2024.
